In [1]:
from __future__ import annotations

import functools

from substrateinterface import SubstrateInterface

from scripts.utils.chain_model import Chain, ChainAsset
from scripts.utils.chain_model import NativeAssetType, StatemineAssetType, UnsupportedAssetType, OrmlAssetType
from utils.work_with_data import get_data_from_file

C:\Users\Valentun\PycharmProjects\fearless-utils\venv\Lib\site-packages\eth_utils\network.py:44: UserWarning: Network 345 with name 'Yooldo Verse Mainnet' does not have a valid ChainId. eth-typing should be updated with the latest networks.
  warnings.warn(
C:\Users\Valentun\PycharmProjects\fearless-utils\venv\Lib\site-packages\eth_utils\network.py:44: UserWarning: Network 12611 with name 'Astar zkEVM' does not have a valid ChainId. eth-typing should be updated with the latest networks.
  warnings.warn(


In [2]:
debug_log_enabled = False

def debug_log(message: str):
    if debug_log_enabled:
        print(message)

In [3]:
from scalecodec import ScaleBytes, Tuple as ScaleTuple


# This is a modified implementation of Tuple.process_encode that fixes issue https://github.com/polkascan/py-scale-codec/issues/126
def process_encode(self: ScaleTuple, value):
    data = ScaleBytes(bytearray())
    self.value_object = ()

    # CHANGED: we additionally check for len(self.type_mapping) == 1 when wrapping the value 
    if len(self.type_mapping) == 1 or type(value) not in (list, tuple):
        value = [value]

    if len(value) != len(self.type_mapping):
        raise ValueError('Element count of value ({}) doesn\'t match type_definition ({})'.format(
            len(value), len(self.type_mapping))
        )

    for idx, member_type in enumerate(self.type_mapping):
        element_obj = self.runtime_config.create_scale_object(
            member_type, metadata=self.metadata
        )
        data += element_obj.encode(value[idx])
        self.value_object += (element_obj,)

    return data


ScaleTuple.process_encode = process_encode

In [4]:
def dry_run_api_ext(runtime_types_prefix: str | None) -> dict | None:
    if runtime_types_prefix is None:
        return None

    return {
        "runtime_api": {
            "DryRunApi": {
                "methods": {
                    "dry_run_call": {
                        "description": "Dry run runtime call",
                        "params": [
                            {
                                "name": "origin_caller",
                                "type": f"{runtime_types_prefix}::OriginCaller"
                            },
                            {
                                "name": "call",
                                "type": "GenericCall"
                            }
                        ],
                        "type": "CallDryRunEffectsResult"
                    },
                    "dry_run_xcm": {
                        "description": "Dry run xcm program",
                        "params": [
                            {
                                "name": "origin_location",
                                "type": "xcm::VersionedLocation"
                            },
                            {
                                "name": "xcm",
                                "type": "xcm::VersionedXcm"
                            }
                        ],
                        "type": "XcmDryRunEffectsResult"
                    },
                },
                "types": {
                    "CallDryRunEffectsResult": {
                        "type": "enum",
                        "type_mapping": [
                            [
                                "Ok",
                                "CallDryRunEffects"
                            ],
                            [
                                "Error",
                                "DryRunEffectsResultErr"
                            ]
                        ]
                    },
                    "DryRunEffectsResultErr": {
                        "type": "enum",
                        "value_list": [
                            "Unimplemented",
                            "VersionedConversionFailed"
                        ]
                    },
                    "CallDryRunEffects": {
                        "type": "struct",
                        "type_mapping": [
                            [
                                "execution_result",
                                "CallDryRunExecutionResult"
                            ],
                            [
                                "emitted_events",
                                f"Vec<{runtime_types_prefix}::RuntimeEvent>"
                            ],
                            [
                                "local_xcm",
                                "Option<xcm::VersionedXcm>"
                            ],
                            [
                                "forwarded_xcms",
                                "Vec<(xcm::VersionedLocation, Vec<xcm::VersionedXcm>)>"
                            ]
                        ]
                    },
                    "CallDryRunExecutionResult": {
                        "type": "enum",
                        "type_mapping": [
                            [
                                "Ok",
                                "DispatchPostInfo"
                            ],
                            [
                                "Error",
                                "DispatchErrorWithPostInfo"
                            ]
                        ]
                    },
                    "XcmDryRunEffects": {
                        "type": "struct",
                        "type_mapping": [
                            [
                                "execution_result",
                                "staging_xcm::v4::traits::Outcome"
                            ],
                            [
                                "emitted_events",
                                f"Vec<{runtime_types_prefix}::RuntimeEvent>"
                            ],
                            [
                                "forwarded_xcms",
                                "Vec<(xcm::VersionedLocation, Vec<xcm::VersionedXcm>)>"
                            ]
                        ]
                    },
                    "XcmDryRunEffectsResult": {
                        "type": "enum",
                        "type_mapping": [
                            [
                                "Ok",
                                "XcmDryRunEffects"
                            ],
                            [
                                "Error",
                                "DryRunEffectsResultErr"
                            ]
                        ]
                    },
                    "DispatchErrorWithPostInfo": {
                        "type": "struct",
                        "type_mapping": [
                            [
                                "post_info",
                                "DispatchPostInfo"
                            ],
                            [
                                "error",
                                "sp_runtime::DispatchError"
                            ],
                        ]
                    },
                    "DispatchPostInfo": {
                        "type": "struct",
                        "type_mapping": [
                            [
                                "actual_weight",
                                "Option<Weight>"
                            ],
                            [
                                "pays_fee",
                                "frame_support::dispatch::Pays"
                            ],
                        ]
                    }
                }
            },
        }
    }

In [5]:
from dataclasses import dataclass
import types
from abc import ABC, abstractmethod
from substrateinterface.utils.ss58 import ss58_decode
from typing import Union, List, Self, Callable, TypeVar

xcm_version_mode_consts = types.SimpleNamespace()
xcm_version_mode_consts.AlreadyVersioned = "AlreadyVersioned"
xcm_version_mode_consts.DefaultVersion = "DefaultVersion"


class VerionsedXcm:
    unversioned: dict
    versioned: dict | List
    version: int

    @staticmethod
    def default_xcm_version() -> int:
        return 4

    def __init__(self,
                 message: dict | List,
                 message_version: str | int = xcm_version_mode_consts.DefaultVersion,
                 ):
        match message_version:
            case int():
                self._init_from_unversioned(message, version=message_version)
            case xcm_version_mode_consts.AlreadyVersioned:
                self._init_from_versioned(message)
            case xcm_version_mode_consts.DefaultVersion:
                self._init_from_unversioned(message, version=None)
            case _:
                raise Exception(f"Unknown message version mode: {message_version}")

    def _init_from_versioned(self, message: dict):
        if type(message) is not dict:
            raise Exception(f"Already versioned xcm must be a dict with a single version key, got: {message}")

        version_key = next(iter(message))

        self.version = self._parse_version(version_key)
        self.versioned = message
        self.unversioned = message[version_key]

    def _init_from_unversioned(self, message: dict | List, version: int | None):
        if version is None:
            self.version = VerionsedXcm.default_xcm_version()
        else:
            self.version = version

        self.unversioned = message
        self.versioned = {f"V{self.version}": message}

    @staticmethod
    def _parse_version(version_key: str):
        return int(version_key.removeprefix("V"))

    def __str__(self):
        return str(self.versioned)

    def is_v4(self) -> bool:
        return self.version == 4


def multi_location(parents: int, junctions: List[dict]) -> VerionsedXcm:
    interior = "Here"

    if len(junctions) > 0:
        interior_variant = f"X{len(junctions)}"

        if VerionsedXcm.default_xcm_version() <= 3 and len(junctions) == 1:
            interior = {interior_variant: junctions[0]}
        else:
            interior = {interior_variant: junctions}

    return VerionsedXcm({"parents": parents, "interior": interior})


def asset_id(location: VerionsedXcm) -> VerionsedXcm:
    if location.is_v4():
        return location
    else:
        return VerionsedXcm({"Concrete": location.unversioned})


def asset(location: VerionsedXcm, amount: int) -> VerionsedXcm:
    return VerionsedXcm({"id": asset_id(location).unversioned, "fun": {"Fungible": amount}})


def assets(location: VerionsedXcm, amount: int) -> VerionsedXcm:
    return VerionsedXcm([asset(location, amount).unversioned])


def absolute_location(junctions: List[dict]) -> VerionsedXcm:
    return multi_location(parents=0, junctions=junctions)


def xcm_program(message: List[dict] | dict[str, List]) -> VerionsedXcm:
    # List of instructions
    if type(message) is list:
        return VerionsedXcm(message)
    # Versioned program
    else:
        return VerionsedXcm(message, message_version=xcm_version_mode_consts.AlreadyVersioned)


def buy_execution(fees_asset: VerionsedXcm, weight_limit: str | dict = "Unlimited"):
    return {'BuyExecution': {'fees': fees_asset.unversioned, 'weight_limit': weight_limit}}


def deposit_asset(
        beneficiary: str,
        evm: bool
):
    return {"DepositAsset": {"assets": {'Wild': {'AllCounted': 1}},
                             "beneficiary": absolute_location(
                                 junctions=[account_junction(beneficiary, evm)]).unversioned}}


def account_junction(
        account: str,
        evm: bool,
) -> dict:
    account_id = decode_account_id(account)

    if evm:
        return {"AccountKey20": {"network": None, "key": account_id}}
    else:
        return {"AccountId32": {"network": None, "id": account_id}}


def decode_account_id(address: str) -> str:
    if address.startswith("0x"):
        return address.lower()
    else:
        return "0x" + ss58_decode(address)


def parachain_junction(parachain_id: int) -> dict:
    return {"Parachain": parachain_id}


cacheMissing = object()


class ReserveLocation:
    id: str

    _reserve_chain: str
    _junctions: List[dict]
    _registry: XcmRegistry

    _cached_parachain_id: int | None | cacheMissing = cacheMissing

    def __init__(
            self,
            id: str,
            reserve_chain_id: str,
            junctions: List[dict],
            registry: XcmRegistry
    ):
        self.id = id
        self._reserve_chain_id = reserve_chain_id
        self._junctions = junctions
        self._registry = registry

    def parachain_id(self) -> int | None:
        if self._cached_parachain_id != cacheMissing:
            return self._cached_parachain_id

        computed_value = self._compute_parachain_id()
        self._cached_parachain_id = computed_value

        return computed_value

    def reanchor(self, pov_chain: XcmChain) -> VerionsedXcm:
        reserve_asset_parachain_id = self.parachain_id()

        same_chain = reserve_asset_parachain_id == pov_chain.parachain_id

        if same_chain:
            return multi_location(parents=0, junctions=self._local_junctions())
        else:
            return multi_location(parents=1, junctions=self._junctions)

    def reserve_chain(self) -> XcmChain:
        return self._registry.get_chain(self._reserve_chain_id)
    
    def reserve_chain_id(self) -> str:
        return self._reserve_chain_id

    def _local_junctions(self):
        if self.parachain_id() is not None:
            return self._junctions[1:]
        else:
            return self._junctions

    def _compute_parachain_id(self) -> int | None:
        if len(self._junctions) == 0:
            return None

        first_junction = self._junctions[0]

        if "Parachain" in first_junction:
            return first_junction["Parachain"]

        return None


class ReserveLocations:
    _locations_by_reserve_id: dict[str, ReserveLocation]
    
    # By default, asset reserve id is equal to its symbol
    # This mapping allows to override that for cases like multiple reserves (Statemine & Polkadot for DOT)
    _asset_reserve_override: dict[Tuple[str, int], str]
    
    def __init__(
            self,
            locations_by_reserve_id: dict[str, ReserveLocation],
            asset_reserve_override: dict[Tuple[str, int], str],
    ):
        self._locations_by_reserve_id = locations_by_reserve_id
        self._asset_reserve_override = asset_reserve_override
        
        
    def get_reserve(self, chain_asset: ChainAsset):
        symbol = chain_asset.unified_symbol()
                
        override = self._asset_reserve_override.get(chain_asset.full_chain_asset_id(), None)
        
        reserve_id = override if override is not None else symbol

        return self._locations_by_reserve_id[reserve_id]


@dataclass
class Parachain:
    parachain_id: int
    chain: Chain


class XcmRegistry:
    reserves: ReserveLocations
    relay: XcmChain
    parachains: List[XcmChain]

    _chains_by_id: dict[str, XcmChain]
    _parachains_by_para_id: dict[int, XcmChain]

    def __init__(
            self,
            relay: Chain,
            parachains: List[Parachain],
            reserves_constructor: Callable[[XcmRegistry], ReserveLocations],
    ):
        self.reserves = reserves_constructor(self)
        self.relay = XcmChain(relay, parachain_id=None, registry=self)
        self.parachains = [XcmChain(parachain.chain, parachain_id=parachain.parachain_id, registry=self) for parachain
                           in parachains]

        self._chains_by_id = self._associate_chains_by_id([self.relay] + self.parachains)
        self._parachains_by_para_id = self._associate_parachains_by_id(self.parachains)

    def get_parachain(self, parachain_id: int | None) -> XcmChain:
        if parachain_id is None:
            return self.relay

        return self._parachains_by_para_id[parachain_id]

    def get_chain(self, chain_id: str) -> XcmChain:
        return self._chains_by_id[chain_id]
    
    def has_chain(self, chain_id: str) -> bool:
        return chain_id in self._chains_by_id

    @staticmethod
    def _associate_chains_by_id(all_chains: List[XcmChain]) -> dict[str, XcmChain]:
        return {xcm_chain.chain.chainId: xcm_chain for xcm_chain in all_chains}

    @staticmethod
    def _associate_parachains_by_id(all_parachains: List[XcmChain]) -> dict[int, XcmChain]:
        return {xcm_chain.parachain_id: xcm_chain for xcm_chain in all_parachains}


xcm_pallet_aliases = ["PolkadotXcm", "XcmPallet"]

T = TypeVar('T')


class TransferType(ABC):

    @abstractmethod
    def check_remote_reserve(self) -> Union['XcmChain', None]:
        pass

    @abstractmethod
    def transfer_type_call_param(self) -> dict | str:
        pass
    
    def __str__(self):
        return self.transfer_type_call_param()


class Teleport(TransferType):

    def check_remote_reserve(self) -> Union['XcmChain', None]:
        return None

    def transfer_type_call_param(self) -> dict | str:
        return "Teleport"


class LocalReserve(TransferType):

    def check_remote_reserve(self) -> Union['XcmChain', None]:
        return None

    def transfer_type_call_param(self) -> dict | str:
        return "LocalReserve"


class DestinationReserve(TransferType):

    def check_remote_reserve(self) -> Union['XcmChain', None]:
        return None

    def transfer_type_call_param(self) -> dict | str:
        return "DestinationReserve"


class RemoteReserve(TransferType):
    _origin_chain: 'XcmChain'
    _reserve_chain: 'XcmChain'
    _registry: XcmRegistry

    def __init__(self, origin_chain: 'XcmChain', reserve_chain: 'XcmChain', registry: 'XcmRegistry'):
        self._reserve_chain = reserve_chain
        self._registry = registry
        self._origin_chain = origin_chain

    def check_remote_reserve(self) -> Union['XcmChain', None]:
        return self._reserve_chain

    def transfer_type_call_param(self) -> dict | str:
        return {"RemoteReserve": self._origin_chain.sibling_location_of(self._reserve_chain).versioned}

class XcmChain:
    chain: Chain
    parachain_id: int | None
    _registry: XcmRegistry

    def __init__(
            self,
            chain: Chain,
            parachain_id: int | None,
            registry: XcmRegistry,
    ):
        self.chain = chain
        self._registry = registry
        self.parachain_id = parachain_id

    def access_substrate(self, action: Callable[[SubstrateInterface], T]) -> T:
        return self.chain.access_substrate(action)

    def sibling_location_of(self, destination_chain: Self) -> VerionsedXcm:
        parents = 1 if self.parachain_id is not None else 0

        junctions = []

        if destination_chain.parachain_id is not None:
            junctions.append(parachain_junction(destination_chain.parachain_id))

        return multi_location(parents=parents, junctions=junctions)

    def account_location(self, account: str):
        return multi_location(parents=0, junctions=[account_junction(account, evm=self.chain.has_evm_addresses())])

    def relative_reserve_location_of(self, chainAsset: ChainAsset) -> VerionsedXcm:
        reserve = self._registry.reserves.get_reserve(chainAsset)
        return reserve.reanchor(pov_chain=self)

    def xcm_pallet_alias(self) -> str:
        result = next((candidate for candidate in xcm_pallet_aliases if
                       self.access_substrate(lambda s: s.get_metadata_module(candidate)) is not None), None)

        if result is None:
            raise Exception(f"No XcmPallet or its aliases has been found. Searched aliases: {xcm_pallet_aliases}")

        return result

    def is_system_parachain(self) -> bool:
        return self.parachain_id is not None and 1000 <= self.parachain_id < 2000

    def is_relay(self) -> bool:
        return self.parachain_id is None

    def transfer_type(self, destination_chain: Self, chain_asset: ChainAsset) -> TransferType:
        reserve = self._registry.reserves.get_reserve(chain_asset)
        reserve_parachain_id = reserve.parachain_id()

        if self._should_use_teleport(destination_chain):
            return Teleport()
        elif self.parachain_id == reserve_parachain_id:
            return LocalReserve()
        elif destination_chain.parachain_id == reserve_parachain_id:
            return DestinationReserve()
        else:
            reserve_chain = reserve.reserve_chain()
            return RemoteReserve(origin_chain=self, reserve_chain=reserve_chain, registry=reserve_chain)

    def _should_use_teleport(self, destination_chain: Self) -> bool:
        to_relay_teleport = self.is_system_parachain() and destination_chain.is_relay()
        from_relay_teleport = self.is_relay() and destination_chain.is_system_parachain()

        return to_relay_teleport or from_relay_teleport




In [6]:
def map_junction_from_config(config_key: str, config_value) -> dict:
    match config_key:
        case "parachainId":
            return {"Parachain": config_value}
        case "generalKey":
            return {"GeneralKey": general_key_junction(config_value) }
        case "generalIndex":
            return {"GeneralIndex": int(config_value)}
        case "palletInstance":
            return {"PalletInstance": config_value}

def truncate_or_pad(data: bytes, required_size: int) -> bytes:
    truncated = data[0::required_size]
    padded = truncated.rjust(required_size,  b'\0')
    return bytes(padded)
    
def general_key_junction(key) -> dict:
    scale_bytes = ScaleBytes(key)
    
    fixed_size_bytes = truncate_or_pad(scale_bytes.data, required_size=32)
    
    return { "length": scale_bytes.length, "data": fixed_size_bytes }


def build_reserve_locations_map(
        asset_locations_config: dict,
        xcm_registry: XcmRegistry
) -> dict[str, List[ReserveLocation]]:
    result = {}
    
    for reserve_id, reserve_config in asset_locations_config.items():    
        junctions = [map_junction_from_config(config_key, config_value) for config_key, config_value in
                     reserve_config["multiLocation"].items()]
        
        chain_id = reserve_config["chainId"]
        
        result[reserve_id] = ReserveLocation(reserve_id, chain_id, junctions, xcm_registry)
        
    return result

def build_reserve_overrides(chain_transfers_config: dict) -> dict[Tuple[str, int], str]:
    result = {}
    
    for xcm_chain_config in chain_transfers_config:
        origin_chain_id = xcm_chain_config["chainId"]
        
        for origin_asset_config in xcm_chain_config["assets"]:
            origin_asset_id = origin_asset_config["assetId"]
            origin_asset_location = origin_asset_config.get("assetLocation", None)
            if origin_asset_location is not None:
                result[(origin_chain_id, origin_asset_id)] = origin_asset_location

    return result

def build_reserves(xcm_config: dict, xcm_registry: XcmRegistry) -> ReserveLocations:
    reserve_locations = build_reserve_locations_map(xcm_config["assetsLocation"], xcm_registry)
    asset_reserve_overrides = build_reserve_overrides(xcm_config["chains"])
    
    return ReserveLocations(reserve_locations, asset_reserve_overrides)


In [7]:
polkadot_id = "91b171bb158e2d3848fa23a9f1c25182fb8e20313b2c1eb49219da7a70ce90c3"
polkadot_asset_hub_id = "68d56f15f85d3136970ec16946040bc1752654e906147f7e43e9d539d7c3de2f"
polkadot_bridge_hub_id = "dcf691b5a3fbe24adc99ddc959c0561b973e329b1aef4c4b22e7bb2ddecb4464"
moonbeam_id = "fe58ea77779b7abda7da4ec526d14db9b1e9cd40a217c34892af80a9b332b76d"
bifrost_id = "262e1b2ad728475fd6fe88e62d34c200abe6fd693931ddad144059b1eb884e5b"
astar_id = "9eb76c5184c4ab8679d2d5d819fdf90b9c001403e9e17da2e14b6d8aec4029c6"

additional_xcm_data = get_data_from_file("xcm_additional_data.json")
chains_file = get_data_from_file("../chains/v21/chains_dev.json")

relay: Chain | None = None
parachains = []

for chain_config in chains_file:
    additional_xcm_chain_data = additional_xcm_data.get(chain_config["chainId"], None)

    if additional_xcm_chain_data is None:
        debug_log(f"No additional xcm data found for {chain_config["name"]}, skipping")
        continue

    runtime_prefix = additional_xcm_chain_data["runtimePrefix"]
    type_registry = dry_run_api_ext(runtime_prefix)
    chain = Chain(chain_config, type_registry)

    if chain.chainId == polkadot_id:
        relay = chain
    else:
        parachain = Parachain(additional_xcm_chain_data["parachainId"], chain)
        parachains.append(parachain)

if relay is None:
    raise Exception("Relay was not found in configuration")

xcm_config = get_data_from_file("../xcm/v7/transfers_dev.json")

registry = XcmRegistry(relay, parachains, functools.partial(build_reserves, xcm_config))

polkadot = registry.relay
polkadot_asset_hub = registry.get_chain(polkadot_asset_hub_id)
moonbeam = registry.get_chain(moonbeam_id)
bifrost = registry.get_chain(bifrost_id)
astar = registry.get_chain(astar_id)


In [8]:
from scalecodec import GenericCall, GenericEvent


class XcmSentEvent:
    _attributes: dict
    sent_message: VerionsedXcm

    def __init__(self, event_data: dict):
        self._attributes = event_data["attributes"]
        self.sent_message = xcm_program(self._attributes["message"])


def is_call_run_error(execution_result: dict):
    return "Error" in execution_result

def extract_dispatch_error_message(chain: XcmChain, dispatch_error) -> str:
    error_description: str

    if "Module" in dispatch_error:
        module_error = dispatch_error["Module"]
        error_index = int(module_error["error"][2:4], 16)

        error = chain.access_substrate(
            lambda s: s.metadata.get_module_error(module_index=module_error["index"], error_index=error_index))
        error_description = str(error)
    else:
        error_description = str(dispatch_error)

    return error_description

def handle_call_run_error_execution_result(chain: XcmChain, execution_result: dict):
    error = execution_result["Error"]["error"]
    error_description = extract_dispatch_error_message(chain, error)
    
    raise Exception(error_description)

def find_event(events: List, event_module: str, event_name: str) -> dict | None:
    matching_events = find_events(events, event_module, event_name)
    return next(iter(matching_events), None)

def find_events(events: List, event_module: str, event_name: str) -> List[dict]:
    return [event for event in events if event["module_id"] == event_module and event["event_id"] == event_name]

def find_xcm_sent_event(chain: XcmChain, events: List) -> XcmSentEvent | None:
    event_data = find_event(events, event_module=chain.xcm_pallet_alias(), event_name="Sent")
    if event_data is not None:
        return XcmSentEvent(event_data)
    else:
        return None


def find_forwarded_xcm(
        success_dry_run_effects: dict,
        final_destination_account: str,
) -> VerionsedXcm | None:
    forwarded_xcms = success_dry_run_effects["forwarded_xcms"]

    final_destination_account_id = decode_account_id(final_destination_account)

    for (destination, messages) in forwarded_xcms:
        for message in messages:
            message_program = xcm_program(message)
            extracted_account = extract_final_beneficiary_from_program(message_program)

            if extracted_account == final_destination_account_id:
                return message_program

    return None

def search_for_error_in_events(chain: XcmChain, events: List) -> str | None:
    dispatched_as_events = find_events(events, "Utility", "DispatchedAs")
    
    for dispatched_as_event in dispatched_as_events:   
        dispatch_result = dispatched_as_event["attributes"]["result"]
    
        if "Err" not in dispatch_result:
            continue
    
        error_description = extract_dispatch_error_message(chain, dispatch_result["Err"])
        return error_description
    
    return None


def find_sent_xcm(
        origin: XcmChain,
        success_dry_run_effects: dict,
        final_destination_account: str
) -> VerionsedXcm:
    emitted_events = success_dry_run_effects["emitted_events"]
    xcm_sent_event = find_xcm_sent_event(origin, emitted_events)
    if xcm_sent_event is not None:
        debug_log(f"Found sent xcm in XcmSent event")
        return xcm_sent_event.sent_message

    forwarded_xcm = find_forwarded_xcm(success_dry_run_effects, final_destination_account)
    if forwarded_xcm is not None:
        debug_log(f"Found sent xcm in forwarded xcms")
        return forwarded_xcm
    
    error = search_for_error_in_events(origin, emitted_events)
    if error is not None:
        raise Exception(f"Execution failed with {error}")

    raise Exception(f"Sent xcm was not found, got: {success_dry_run_effects}")


def extract_destination_asset(xcm_message: List[dict]) -> dict:
    first_instruction = xcm_message[0]

    if "ReceiveTeleportedAsset" in first_instruction:
        return first_instruction["ReceiveTeleportedAsset"][0]
    elif "ReserveAssetDeposited" in first_instruction:
        return first_instruction["ReserveAssetDeposited"][0]

    raise Exception("Found no destination assets")


def extract_final_beneficiary_from_program(program: VerionsedXcm) -> str | None:
    for instruction in program.unversioned:
        from_instruction = extract_final_beneficiary_from_instruction(instruction)
        if from_instruction is not None:
            return from_instruction

    return None


def get_single_key(instruction: dict | str) -> str | None:
    if type(instruction) is str:
        return None

    if len(instruction) != 1:
        raise Exception(f"Expected a single key dict, got: {instruction}")

    return next(iter(instruction))


def extract_final_beneficiary_from_instruction(instruction: dict) -> str | None:
    match get_single_key(instruction):
        case "DepositAsset":
            return extract_beneficiary_from_location(instruction["DepositAsset"]["beneficiary"])
        case "DepositReserveAsset" | "InitiateReserveWithdraw" | "InitiateTeleport" | "TransferReserveAsset" as key:
            nested_message = instruction[key]["xcm"]
            return extract_final_beneficiary_from_program(xcm_program(nested_message))
        case _:
            return None


def extract_beneficiary_from_location(location: dict) -> str | None:
    interior = location["interior"]
    x1_junction: dict

    match get_single_key(interior):
        # Accounts are always X1
        case "X1":
            x1_junction = interior["X1"]
            # pre-v3 x1 interior is just junction itself, otherwise it is a list of single element 
            if type(x1_junction) is list:
                x1_junction = x1_junction[0]
        case _:
            return None

    match get_single_key(x1_junction):
        case "AccountKey20":
            return x1_junction["AccountKey20"]["key"]
        case "AccountId32":
            return x1_junction["AccountId32"]["id"]
        case _:
            return None


def signed_origin(account: str) -> dict:
    return {"system": {"Signed": account}}


def root_origin() -> dict:
    return {"system": "Root"}


def dry_run_xcm_call(
        chain: XcmChain,
        call: GenericCall,
        origin: dict,
        final_destination_account: str
) -> VerionsedXcm:
    dry_run_result = chain.access_substrate(
        lambda substrate: substrate.runtime_call(api="DryRunApi", method="dry_run_call",
                                                 params={"origin_caller": origin, "call": call})).value

    dry_run_effects = dry_run_result["Ok"]
    execution_result = dry_run_effects["execution_result"]

    if is_call_run_error(execution_result):
        handle_call_run_error_execution_result(chain, execution_result)
    else:
        return find_sent_xcm(chain, dry_run_effects, final_destination_account)


def is_xcm_run_error(execution_result: dict):
    return "Incomplete" in execution_result or "Error" in execution_result


def handle_xcm_run_error_execution_result(execution_result: dict):
    if "Incomplete" in execution_result:
        error = execution_result["Incomplete"]["error"]
    elif "Error" in execution_result:
        error = execution_result["Error"]["error"]
    else:
        error = execution_result

    raise Exception(str(error))


def dry_run_xcm(chain: XcmChain, xcm: VerionsedXcm, origin: VerionsedXcm) -> dict:
    dry_run_effects = chain.access_substrate(
        lambda substrate: substrate.runtime_call(api="DryRunApi", method="dry_run_xcm",
                                                 params={"origin_location": origin.versioned,
                                                         "xcm": xcm.versioned})).value["Ok"]
    execution_result = dry_run_effects["execution_result"]

    if is_xcm_run_error(execution_result):
        handle_xcm_run_error_execution_result(execution_result)
    else:
        return dry_run_effects


def dry_run_intermediate_xcm(
        chain: XcmChain,
        xcm: VerionsedXcm,
        origin: VerionsedXcm,
        final_destination_account: str
) -> VerionsedXcm:
    dry_run_effects = dry_run_xcm(chain, xcm, origin)

    return find_sent_xcm(chain, dry_run_effects, final_destination_account)


def dry_run_final_xcm(chain: XcmChain, xcm: VerionsedXcm, origin: VerionsedXcm) -> List[GenericEvent]:
    dry_run_effects = dry_run_xcm(chain, xcm, origin)

    return dry_run_effects["emitted_events"]


def multi_address(account: str, evm: bool):
    if evm:
        return account
    else:
        return {"Id": account}


def compose_dispatch_as(
        substrate: SubstrateInterface,
        origin: dict,
        call: GenericCall
):
    return substrate.compose_call(
        call_module="Utility",
        call_function="dispatch_as",
        call_params={
            "as_origin": origin,
            "call": call
        }
    )


def compose_native_fund(
        substrate: SubstrateInterface,
        chain: XcmChain,
        account: str,
        amount_planks: int,
) -> GenericCall:
    return substrate.compose_call(
        call_module="Balances",
        call_function="force_set_balance",
        call_params={
            "who": multi_address(account, chain.chain.has_evm_addresses()),
            "new_free": amount_planks
        }
    )


def compose_assets_fund(
        substrate: SubstrateInterface,
        chain: XcmChain,
        statemine_type: StatemineAssetType,
        account: str,
        amount_planks: int,
) -> GenericCall:
    asset_info = substrate.query(module=statemine_type.pallet_name(), storage_function="Asset",
                                 params=[statemine_type.encodable_asset_id()]).value
    issuer = asset_info["issuer"]

    mint_call = substrate.compose_call(
        call_module=statemine_type.pallet_name(),
        call_function="mint",
        call_params={
            "id": statemine_type.encodable_asset_id(),
            "beneficiary": multi_address(account, chain.chain.has_evm_addresses()),
            "amount": amount_planks
        }
    )

    return compose_dispatch_as(
        substrate=substrate,
        origin=signed_origin(issuer),
        call=mint_call
    )


def compose_orml_fund(
        substrate: SubstrateInterface,
        chain: XcmChain,
        orml_type: OrmlAssetType,
        account: str,
        amount_planks: int,
) -> GenericCall:
    return substrate.compose_call(
        call_module=orml_type.pallet_name(),
        call_function="set_balance",
        call_params={
            "who": multi_address(account, chain.chain.has_evm_addresses()),
            "currency_id": orml_type.encodable_asset_id(),
            "new_free": amount_planks,
            "new_reserved": 0,
        }
    )


def compose_fund_call(
        substrate: SubstrateInterface,
        chain: XcmChain,
        chain_asset: ChainAsset,
        account: str,
        amount_planks: int,
) -> GenericCall:
    match chain_asset.type:
        case NativeAssetType():
            return compose_native_fund(substrate, chain, account, amount_planks)
        case StatemineAssetType() as statemineType:
            return compose_assets_fund(substrate, chain, statemineType, account, amount_planks)
        case OrmlAssetType() as ormlType:
            return compose_orml_fund(substrate, chain, ormlType, account, amount_planks)

        case UnsupportedAssetType():
            raise Exception("Unsupported asset type")


def fund_account_and_then(
        substrate: SubstrateInterface,
        chain: XcmChain,
        chain_asset: ChainAsset,
        account: str,
        amount: int,
        next_call: GenericCall
) -> GenericCall:
    planks_in_sending_asset = chain_asset.planks(amount)
    
    calls = []
    
    fund_sending_asset_call = compose_fund_call(substrate, chain, chain_asset, account, planks_in_sending_asset)
    calls.append(fund_sending_asset_call)
    
    utility_asset = chain.chain.get_utility_asset()
    if utility_asset.id != chain_asset.id:
        planks_in_utility_asset = utility_asset.planks(amount)
        fund_utility_asset_call = compose_fund_call(substrate, chain, utility_asset, account, planks_in_utility_asset)
        calls.append(fund_utility_asset_call)

    wrapped_next_call = compose_dispatch_as(
        substrate=substrate,
        origin=signed_origin(account),
        call=next_call
    )
    calls.append(wrapped_next_call)

    return substrate.compose_call(
        call_module="Utility",
        call_function="batch_all",
        call_params={
            "calls": calls
        }
    )

substrate_account = "13mp1WEs72kbCBF3WKcoK6Hfhu2HHZGpQ4jsKCZbfd6FoRvH"
evm_account = "0x0c7485f4AA235347BDE0168A59f6c73C7A42ff2C"

def dry_run_account_for_chain(chain: XcmChain) -> str:
    if chain.chain.has_evm_addresses():
        return evm_account
    else:
        return substrate_account
    
def get_asset(origin_chain: XcmChain, chain_asset_symbol_or_id: str | int) -> ChainAsset:
    if type(chain_asset_symbol_or_id) is int:
        return origin_chain.chain.get_asset_by_id(chain_asset_symbol_or_id)
    
    return origin_chain.chain.get_asset_by_symbol(chain_asset_symbol_or_id)

def dry_run_transfer(
        origin_chain_id: str,
        destination_chain_id: str,
        chain_asset_symbol_or_id: str | int,
):   
    origin_chain = registry.get_chain(origin_chain_id)
    destination_chain = registry.get_chain(destination_chain_id)
    
    chain_asset = get_asset(origin_chain, chain_asset_symbol_or_id)
    
    print(f"Dry running transfer of {chain_asset.symbol} from {origin_chain.chain.name} to {destination_chain.chain.name}")
    
    amount = 100
    sender = dry_run_account_for_chain(origin_chain)
    recipient = dry_run_account_for_chain(destination_chain)
        
    transfer_type = origin_chain.transfer_type(destination_chain, chain_asset)

    token_location_origin = origin_chain.relative_reserve_location_of(chain_asset)

    dest = origin_chain.sibling_location_of(destination_chain).versioned
    assets_param = assets(token_location_origin, amount=chain_asset.planks(amount)).versioned

    remote_reserve_chain = transfer_type.check_remote_reserve()

    beneficiary = destination_chain.account_location(recipient).versioned

    fee_asset_item = 0
    weight_limit = "Unlimited"

    debug_log(f"{transfer_type=}")
    debug_log(f"{dest=}")
    debug_log(f"{beneficiary=}")
    debug_log(f"{assets_param=}")
    debug_log(f"{fee_asset_item=}")
    debug_log(f"{weight_limit=}")

    debug_log("\n------------------\n")

    def transfer_assets_call(substrate: SubstrateInterface) -> GenericCall:
        return substrate.compose_call(
            call_module=origin_chain.xcm_pallet_alias(),
            call_function="transfer_assets",
            call_params={
                "dest": dest,
                "assets": assets_param,
                "beneficiary": beneficiary,
                "fee_asset_item": fee_asset_item,
                "weight_limit": weight_limit
            }
        )

    call = origin_chain.access_substrate(
        lambda s: fund_account_and_then(s, origin_chain, chain_asset, account=sender, amount=amount*10,
                                        next_call=transfer_assets_call(s))
    )

    message_to_next_hop = dry_run_xcm_call(origin_chain, call, root_origin(), final_destination_account=recipient)
    debug_log(f"Transfer successfully initiated on {origin_chain.chain.name}, message: {message_to_next_hop}")
    debug_log("\n------------------\n\n")

    message_to_destination: VerionsedXcm
    origin_on_destination: VerionsedXcm

    if remote_reserve_chain is not None:
        message_to_reserve = message_to_next_hop
        origin_on_reserve = remote_reserve_chain.sibling_location_of(origin_chain)

        debug_log(f"{origin_on_reserve.versioned=}")

        message_to_destination = dry_run_intermediate_xcm(chain=remote_reserve_chain, xcm=message_to_reserve,
                                                          origin=origin_on_reserve, final_destination_account=recipient)
        debug_log(
            f"Transfer successfully handled by reserve chain {remote_reserve_chain.chain.name}, message: {message_to_reserve.unversioned}\n")

        origin_on_destination = destination_chain.sibling_location_of(remote_reserve_chain)
    else:
        origin_on_destination = destination_chain.sibling_location_of(origin_chain)
        message_to_destination = message_to_next_hop

    debug_log("\n------------------\n")

    destination_events = dry_run_final_xcm(destination_chain, message_to_destination, origin_on_destination)

    debug_log(f"Transfer successfully finished on {destination_chain.chain.name}, final events: {destination_events}")

In [9]:
total_destinations = 0
destination_indices = {}
current_index = 1

def destination_key(origin_chain_id: str, origin_chain_asset: str, destination_chain_id: str) -> str:
    return f"{origin_chain_id}-{origin_chain_asset}-{destination_chain_id}"

for xcm_chain_config in xcm_config["chains"]:
    origin_chain_id = xcm_chain_config["chainId"]
    
    for origin_asset_config in xcm_chain_config["assets"]:
        origin_asset_id = origin_asset_config["assetId"]
        
        total_destinations += len(origin_asset_config["xcmTransfers"])
        
        for transfer_config in origin_asset_config["xcmTransfers"]:
            destination_config = transfer_config["destination"]
            destination_chain_id = destination_config["chainId"]
            
            destination_indices[destination_key(origin_chain_id, origin_asset_id, destination_chain_id)] = current_index
            current_index+=1

In [10]:
chains_with_no_dry_run = set()

start_index = 221
            

for xcm_chain_config in xcm_config["chains"]: 
    origin_chain_id = xcm_chain_config["chainId"]
    
    if not registry.has_chain(origin_chain_id):
        continue
        

    origin_chain = registry.get_chain(origin_chain_id)
    
    for origin_asset_config in xcm_chain_config["assets"]:
        origin_asset_id = origin_asset_config["assetId"]
        origin_chain_asset = origin_chain.chain.get_asset_by_id(origin_asset_id)
        
        reserves = registry.reserves.get_reserve(origin_chain_asset)
        if not registry.has_chain(reserves.reserve_chain_id()):
            continue
        
        for transfer_config in origin_asset_config["xcmTransfers"]: 
            destination_config = transfer_config["destination"]
            destination_chain_id = destination_config["chainId"]
            
            if not registry.has_chain(destination_chain_id):
                continue
            
            current_index = destination_indices[destination_key(origin_chain_id, origin_asset_id, destination_chain_id)]
            if start_index is not None and start_index > current_index:
                continue
            
            print(f"{current_index}/{total_destinations}")
                        
            dry_run_transfer(origin_chain_id, destination_chain_id, origin_asset_id)            

221/478
Dry running transfer of USDT from Polkadot Asset Hub to Moonbeam
Connecting to  wss://statemint-rpc.dwellir.com
Connected to  wss://statemint-rpc.dwellir.com
Connecting to  wss://wss.api.moonbeam.network
Connected to  wss://wss.api.moonbeam.network
223/478
Dry running transfer of USDT from Polkadot Asset Hub to Bifrost Polkadot
Connecting to  wss://bifrost-polkadot-rpc.dwellir.com
Connected to  wss://bifrost-polkadot-rpc.dwellir.com
226/478
Dry running transfer of USDT from Polkadot Asset Hub to Astar
Connecting to  wss://astar.public.curie.radiumblock.co/ws
Connected to  wss://astar.public.curie.radiumblock.co/ws
228/478
Dry running transfer of USDC from Polkadot Asset Hub to Moonbeam
317/478
Dry running transfer of GLMR from Bifrost Polkadot to Moonbeam
320/478
Dry running transfer of GLMR from Bifrost Polkadot to Astar
322/478
Dry running transfer of DOT from Bifrost Polkadot to Polkadot
Connecting to  wss://rpc-polkadot.novasama-tech.org
Connected to  wss://rpc-polkadot.nov

In [ ]:
dhrmp_channels_map = polkadot.substrate.query_map(module="Hrmp", storage_function="HrmpChannels")
hrmp_channels = [result[0].value for result in hrmp_channels_map]
hrmp_channels

In [ ]:
chain_id_by_parachain_id = {parachain.substrate.query("ParachainInfo", "ParachainId").value: parachain.chainId for
                            parachain in polkadot_parachains}
chain_id_by_parachain_id

In [ ]:
from typing import List, Dict, Tuple


def construct_chain_graph(
        relay: Chain,
        parachains: List[chains],
        hrmp_channels: List[Tuple[int, int]],
        chain_id_by_parachain_id: Dict[int, str]
) -> Dict[str, List[str]]:
    result: Dict[str, List[str]] = {}

    def add_edge_by_id(origin: str, destination: str):
        edges = result.get(origin)
        if edges is None:
            result[origin] = [destination]
        else:
            edges.append(destination)

    def add_edge_by_chain(origin: Chain, destination: Chain):
        add_edge_by_id(origin.chainId, destination.chainId)

    # relay is accessible from each parachain
    for parachain in parachains:
        add_edge_by_chain(relay, parachain)
        add_edge_by_chain(parachain, relay)

    # add all supported channels    
    for channel in hrmp_channels:
        origin = chain_id_by_parachain_id.get(channel["recipient"])
        destination = chain_id_by_parachain_id.get(channel["sender"])
        if origin is None or destination is None:
            continue

        add_edge_by_id(origin, destination)

    return result


construct_chain_graph(polkadot, polkadot_parachains, hrmp_channels, chain_id_by_parachain_id)

In [ ]:
polkadot.substrate.runtime_call(api="DryRunApi", method="dry_run_call")